-  reference : https://www.kaggle.com/javalex/nyc-bike-data-analysis-subscribers-and-customers

In [1]:

import numpy as np
import pandas as pd
import geopandas
from shapely.geometry import Point, Polygon

import os 
import sys

import calendar
import glob
import math

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('NYC-BikeShare-2015-2017-combined.csv')

In [3]:
df.head()

,Unnamed: 0,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender,Trip_Duration_in_min
0,0,376,2015-10-01 00:16:26,2015-10-01 00:22:42,3212,Christ Hospital,40.734786,-74.050444,3207,Oakland Ave,40.737604,-74.052478,24470,Subscriber,1960.0,1,6
1,1,739,2015-10-01 00:27:12,2015-10-01 00:39:32,3207,Oakland Ave,40.737604,-74.052478,3212,Christ Hospital,40.734786,-74.050444,24481,Subscriber,1960.0,1,12
2,2,2714,2015-10-01 00:32:46,2015-10-01 01:18:01,3193,Lincoln Park,40.724605,-74.078406,3193,Lincoln Park,40.724605,-74.078406,24628,Subscriber,1983.0,1,45
3,3,275,2015-10-01 00:34:31,2015-10-01 00:39:06,3199,Newport Pkwy,40.728745,-74.032108,3187,Warren St,40.721124,-74.038051,24613,Subscriber,1975.0,1,5
4,4,561,2015-10-01 00:40:12,2015-10-01 00:49:33,3183,Exchange Place,40.716247,-74.033459,3192,Liberty Light Rail,40.711242,-74.055701,24668,Customer,1984.0,0,9


## parameter

In [4]:
year = 2017

#age
age_ranges = ['<20','20-29','30-39','40-49','50-59','60+'] # 각 나이 범위를 구함
age_ranges_limits = [0, 20, 30, 40, 50, 60, np.inf]
age_min = 0
age_max = 100

# trip duration
duration_min = 2 # 2초 이상 사용한 사람이 최소 사용이라고 말할 수 있다
duration_max = 30 * 24 * 60 * 60 # 30일 사용

usertypes = ['all', 'subscriber','customer']

#plotting
font_scale = 1.5


## data validation

### load data :

In [5]:
df = pd.read_csv('NYC-BikeShare-2015-2017-combined.csv')
df.describe()

,Unnamed: 0,Trip Duration,Start Station ID,Start Station Latitude,Start Station Longitude,End Station ID,End Station Latitude,End Station Longitude,Bike ID,Birth Year,Gender,Trip_Duration_in_min
count,735502.000000,7.355020e+05,735502.000000,735502.000000,735502.000000,735502.000000,735502.000000,735502.000000,735502.000000,735502.000000,735502.000000,735502.000000
mean,10582.401084,9.343125e+02,3206.764183,40.723125,-74.046412,3203.387325,40.722371,-74.045415,24914.009133,1979.544239,1.123668,15.570669
std,7679.871929,4.063878e+04,26.546683,0.008121,0.011191,64.358120,0.095296,0.173044,753.768581,9.371192,0.519694,677.312437
min,0.000000,6.100000e+01,3183.000000,40.692640,-74.096937,147.000000,0.000000,-74.096937,14552.000000,1900.000000,0.000000,1.000000
25%,4484.000000,2.460000e+02,3186.000000,40.717732,-74.050656,3186.000000,40.716540,-74.050444,24486.000000,1975.000000,1.000000,4.000000
50%,9156.000000,3.830000e+02,3202.000000,40.721525,-74.044247,3199.000000,40.721124,-74.043845,24602.000000,1982.000000,1.000000,6.000000
75%,15003.000000,6.520000e+02,3211.000000,40.727596,-74.038051,3211.000000,40.727224,-74.036486,24711.000000,1986.000000,1.000000,11.000000
max,34148.000000,2.026021e+07,3426.000000,40.752559,-74.032108,3442.000000,40.801343,0.000000,29296.000000,2000.000000,2.000000,337670.000000


In [7]:
# 시간관련 문자 데이터를 datetime 으로 변경
df['Start time'] = pd.to_datetime(df['Start Time'])
df['Stop Time'] = pd.to_datetime(df['Stop Time'])
df['Birth Year'] = pd.to_numeric(df['Birth Year'], downcast='integer') # 태어난 날을 정수화
df.drop(['Unnamed: 0'], axis=1, inplace=True) # 불필요한 칼럼 삭제
df.head()


,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender,Trip_Duration_in_min,Start time
0,376,2015-10-01 00:16:26,2015-10-01 00:22:42,3212,Christ Hospital,40.734786,-74.050444,3207,Oakland Ave,40.737604,-74.052478,24470,Subscriber,1960,1,6,2015-10-01 00:16:26
1,739,2015-10-01 00:27:12,2015-10-01 00:39:32,3207,Oakland Ave,40.737604,-74.052478,3212,Christ Hospital,40.734786,-74.050444,24481,Subscriber,1960,1,12,2015-10-01 00:27:12
2,2714,2015-10-01 00:32:46,2015-10-01 01:18:01,3193,Lincoln Park,40.724605,-74.078406,3193,Lincoln Park,40.724605,-74.078406,24628,Subscriber,1983,1,45,2015-10-01 00:32:46
3,275,2015-10-01 00:34:31,2015-10-01 00:39:06,3199,Newport Pkwy,40.728745,-74.032108,3187,Warren St,40.721124,-74.038051,24613,Subscriber,1975,1,5,2015-10-01 00:34:31
4,561,2015-10-01 00:40:12,2015-10-01 00:49:33,3183,Exchange Place,40.716247,-74.033459,3192,Liberty Light Rail,40.711242,-74.055701,24668,Customer,1984,0,9,2015-10-01 00:40:12


In [10]:
# 무시해야할 필요가 있는 칼럼을 우선 생성해둠
df["ignore"] = False 
df["ignore_reason"] = ""

In [18]:
# df.shape = (735502,20) 이나 
duplicates = df.duplicated(subset=None, keep='first') # 중복여부 확인, 중복이 있다면 첫번째 값만 남긴다
# duplicates 는 해당 값이 True / False 로 된 값을 가지고 있음

df.insert(len(df.columns), "duplicate", duplicates, allow_duplicates = True)
# df 라는 이름의 dataframe 에 len(df.columns) 번째에 'duplicate' 라는 이름의 칼럼을 넣어라
# 실제 값은 duplicates 를 넣는 것

print("Found {} duplicate rows".format(len(df[duplicates])))
# df[duplicates] 는 df.duplicate = True 인 것만 불러오는 것


Found 62930 duplicate rows


In [23]:
# df bike id, start station id, end station 이 값이 없다면, ignore_reason 칼럼 값에 값 추가
df.loc[df["Bike ID"].isna(), "ignore_reason"] += "Bike ID empty; "
df.loc[df["Start Station ID"].isna(), "ignore_reason"] += "Start Station empty; "
df.loc[df["End Station ID"].isna(), "ignore_reason"] += "End Station empty; "

# 
df.loc[~df["User Type"].isin(["Subscriber", "Customer"]), "ignore_reason"] += "User Type invalid; "


In [25]:
df['Bike ID'].unique()

array([24470, 24481, 24628, 24613, 24668, 24644, 24482, 24550, 24650,
       24584, 24471, 24406, 24723, 24516, 19086, 24561, 24515, 24639,
       24718, 24422, 24452, 24401, 24496, 24391, 24567, 24594, 24558,
       24676, 24483, 24478, 24437, 24615, 24709, 24396, 24666, 24645,
       24582, 24705, 24429, 24540, 24450, 24699, 24469, 24547, 24527,
       24495, 24700, 24535, 24610, 24597, 24443, 24448, 24492, 24498,
       24490, 24633, 24659, 24625, 24510, 24679, 24683, 24669, 24379,
       24556, 24562, 24532, 24590, 24497, 24704, 24579, 24681, 24399,
       24444, 24587, 24557, 24506, 24572, 24620, 24414, 24402, 24434,
       24571, 24606, 24419, 24662, 24647, 24602, 24508, 24593, 24715,
       24420, 24531, 24430, 24581, 24629, 24702, 24409, 24670, 24640,
       24665, 24548, 24455, 24489, 24415, 24618, 24574, 24651, 24658,
       17653, 24513, 24671, 24517, 24608, 24649, 24435, 24504, 24436,
       24570, 24688, 24612, 24493, 24386, 24635, 24595, 24472, 24487,
       24655, 24466,